# The Gear Drop Project

### The Goal

Hello everyone! As you know, our goal is to figure out a simple way to manage the drop system for our game. This means we not only have to manage the rarities of these drops, but what type of weapon drop it is as well. This is my suggestion on how we can easily handle this system.

### What is enum?

Enum is a basic python module that is used as a way to create a set of data for us to use enumeration and other methods on to access that set of data, similar to lists/arrays. It provides easy organization and set up for variables and the value assigned to them, which is why I am showing you this today to help setup our drops.

##### Here is an example of some starting code where we use enum for storing drop rarities and values associated to them

In this example, we will have 7 rarities of gear assigned with the highest level the gear can go to as well. That way, when we use a random number generator later to roll instances of gear drops, we can check that number rolled and use it to drop the correct rarity. For example, 0-450 is a common piece, 451-600 is uncommon, 601-750 is rare, 751-875 is epic, 876-945 is legendary, 946-990 is exalted, and 991-999 is awakened. I also show another way to set up enum, this case being different types of weapon drops

In [1]:
from enum import Enum

# One way to make an enumeration, using a class to easily organise and assign values that we would like
class DropRarities(Enum):
    Common = 450
    Uncommon = 600
    Rare = 750
    Epic = 875
    Legendary = 945
    Exalted = 990
    Awakened = 999

# Another way to make an enumeration using a function, where the values are assigned automatically starting from 0
Weapons = Enum('Weapons', ['Pistol', 'Shotgun', 'SMG', 'Rifle', 'Sniper', 'Grenade Launcher', 'Rocket Launcher'])

I personally prefer the first way, as it looks more organized and is easier to setup, so lets rewrite our gear enum. In cases where we would have very long enums, we would be able to use the auto() function instead of assigning values so that the code itself will increment the values we have assigned to our gear instead of us having to worry about miscounting.

In [2]:
from enum import auto

class Gear(Enum):
    Pistol = auto()
    Shotgun = auto()
    SMG = auto()
    Rifle = auto()
    Sniper = auto()
    Grenade_Launcher = auto()
    Rocket_Launcher = auto()

It is always possible that you would worry about having duplicate values assigned to variables when we do have a smaller enum, such as our DropRarities if we were to add a new rarity like exotic. However, using unique, we would be able to catch these mistakes very quickly.

In [3]:
from enum import unique

@unique
class UpdatedDropRarities(Enum):
    Common = 450
    Uncommon = 600
    Rare = 750
    Epic = 875
    Legendary = 945
    Exalted = 990
    Awakened = 999
    Exotic = 945

ValueError: duplicate values found in <enum 'UpdatedDropRarities'>: Exotic -> Legendary

As you see, when we make an Exotic rarity that happened to have the same cutoff value as our Legendary rarity, it printed out the error and told is exactly which variables in our enum had the same values. This lets us easily go back and figure out which one we need to change.

In [4]:
@unique
class UpdatedDropRarities(Enum):
    Common = 450
    Uncommon = 600
    Rare = 750
    Epic = 875
    Legendary = 945
    Exalted = 990
    Awakened = 999
    Exotic = 970

No issues at all!

### What can we do with enum?

For our use case, enum makes it easier to define rarities and weapon drops. With the first enum being made as a class, we were able to assign rarities with custom numbers, which we can use later on with a random number generator to help determine the rarity of a gear piece. With the second enum being made as a function, we were able to make a list of different weapon types that could drop, where each one has its own value assigned from 1 - 7, and we can use another random number generator to determine that type of weapon drop as well.

You could argue that we could use multiple lists to store our variables and corresponding values, but with enum, we can easily set up it so that if we ever decide to add a new rarity or weapon, we just have to go back to where we declared the enum and add it on wherever we desire.

Before we can move on, we have to figure out how to access our variables and the values assigned to them.

##### Example of Different ways to access our variables

To access a specific rarity or value, we can do the following:

In [5]:
print(DropRarities['Common'].name)
print(DropRarities['Common'].value)

Common
450


In [6]:
print(Gear['Sniper'].name)
print(Gear['Sniper'].value)

Sniper
5


To make it easier to check or access our all of our DropRarities, we can use a for loop.

In [7]:
print("List of rarities and their cutoff value:")
for rarity in DropRarities:
    print("Drops that are of" , rarity.name, "rarity have a max value of", rarity.value)

List of rarities and their cutoff value:
Drops that are of Common rarity have a max value of 450
Drops that are of Uncommon rarity have a max value of 600
Drops that are of Rare rarity have a max value of 750
Drops that are of Epic rarity have a max value of 875
Drops that are of Legendary rarity have a max value of 945
Drops that are of Exalted rarity have a max value of 990
Drops that are of Awakened rarity have a max value of 999


In [8]:
print("List of weapons and their value:")
for weapon in Gear:
    print("Drops that are of" , weapon.name, "class have a value of", weapon.value)

List of weapons and their value:
Drops that are of Pistol class have a value of 1
Drops that are of Shotgun class have a value of 2
Drops that are of SMG class have a value of 3
Drops that are of Rifle class have a value of 4
Drops that are of Sniper class have a value of 5
Drops that are of Grenade_Launcher class have a value of 6
Drops that are of Rocket_Launcher class have a value of 7


We could also print them out as a list if we wanted to, although it wouldn't look pretty

In [9]:
print(list(DropRarities))

[<DropRarities.Common: 450>, <DropRarities.Uncommon: 600>, <DropRarities.Rare: 750>, <DropRarities.Epic: 875>, <DropRarities.Legendary: 945>, <DropRarities.Exalted: 990>, <DropRarities.Awakened: 999>]


In [10]:
print(list(Gear))

[<Gear.Pistol: 1>, <Gear.Shotgun: 2>, <Gear.SMG: 3>, <Gear.Rifle: 4>, <Gear.Sniper: 5>, <Gear.Grenade_Launcher: 6>, <Gear.Rocket_Launcher: 7>]


### Basic implementation of a drop system

Now that we know how to access different variables and their values associated, we can start to code the basics of how a drop system could work.

##### For this example, we will randomly drop the player 3 weapons for the starting tutorial. Ideally, it would be 1 close range weapon, 1 long range weapon, and 1 explosive weapon, but we will show both fully random drops and curated drops

First, we need to import random so that we will be able to randomly generate our rarity and weapon values.

In [11]:
import random

Next, lets make a function to return the rarity of the weapon and type of gun

In [12]:
def determineRarity(rarityValue):
    for rarity in DropRarities:
        if randomRarity < rarity.value:
            return rarity.name
        
def determineGun(weaponValue):
    for weapon in Gear:
        if weaponValue == weapon.value:
            return weapon.name

Now lets drop our player 3 random weapons of any rarity

In [13]:
for drop in range(3):
    randomRarity = random.randint(0, 999)
    randomGun = random.randint(1, 7)
    
    dropRarity = determineRarity(randomRarity)
    dropGun = determineGun(randomGun)
    
    print("You have recieved a", dropRarity, dropGun)

You have recieved a Epic Shotgun
You have recieved a Rare SMG
You have recieved a Rare SMG


Now lets drop our player 3 curated weapons - one close range, one long range, and one explosive weapon that have a max rarity of uncommon

In [14]:
closeGun = determineGun(random.randint(1, 3))
longGun = determineGun(random.randint(4, 5))
explodeGun = determineGun(random.randint(6, 7))
curatedRarities = []
for drop in range(3):
    randomRarity = random.randint(0, 600)
    
    curatedRarities.append(determineRarity(randomRarity))
    
print("You have recieved a", curatedRarities[0], closeGun)
print("You have recieved a", curatedRarities[1], longGun)
print("You have recieved a", curatedRarities[2], explodeGun)

You have recieved a Uncommon Shotgun
You have recieved a Common Rifle
You have recieved a Common Grenade_Launcher


Utilizing our enums and the random integers we generated, we were able to compare and return the proper weapons and their rarities for the game to drop the proper weapons.

### Wrap-Up

As you all can see, enum can be really useful to setup how our drops would work. We can make enums to store our rarities and weapons and easily return in the future to add more weapon or rarity types and assign them values. There isn't a need to worry about enum's getting to long since we are able to use the auto function to make sure we are incrementing our values properly. We can also use unique to make sure when we don't use auto, we aren't accidently assigning the same value to two different variables. We are able to access these variables in many different ways as well, so we can use any way we want depending on the use case. We were able to use enum and compare them with integers just like normal we would with normal variables to drop our players random weapons and rarities as well as curated weapons.